# Declarative data analysis

This notebook demonstrates a "realistic" set of operations and pipeline you might write using functions from `utilz` combined with the `toolz` package. It mimics the declarative style of functional-programming more recently popularized in `R` libraries like `dplyr`. 

In [1]:
import pandas as pd
from utilz import randdf, assign, rename
from toolz import pipe

## Verb-based dataframe manipulation

One of the strengths of libraries like `dplyr` is that it provides a sort of *grammar* for manipulating data. This allows for concise analysis pipelines. `utilz` offers some similar functionality through a few choice *verbs*. Tons of additional functionality is available in the great [plydata](https://plydata.readthedocs.io/en/stable/) library:

In [2]:
# Setup some random data
df = pipe(
    randdf((20, 3)),
    assign(D1=list("abcde") * 4),
    rename({"A1": "rt", "B1": "score", "C1": "speed", "D1": "group"}),
    assign(rt_doubled="rt*2"),
)
df.head()

,rt,score,speed,group,rt_doubled
0,0.835214,0.469788,0.818336,a,1.670428
1,0.933498,0.803162,0.836482,b,1.866995
2,0.870527,0.988006,0.045092,c,1.741054
3,0.331299,0.188222,0.201379,d,0.662598
4,0.060063,0.527094,0.873889,e,0.120126


### Basic slicing/subsetting
`rows` and `cols` behave similarly and can index a Dataframe using *strings*, *lists/arrays*, *tuples* or *ints*:

In [6]:
from utilz import rows, cols

#### strings

`rows` interpret strings as `df.query` expressions, while `cols` looks for a column name:

In [4]:
pipe(df, 
    rows("group == 'c' or group == 'b'"), 
    cols("rt")
    )


,rt
0,0.568678
1,0.277055
2,0.084023
3,0.987609
4,0.533413
5,0.336701
6,0.438272
7,0.480924


Cols can also "negatively index" columns which selects everything *except* the column:

In [5]:
pipe(df, 
    rows("group == 'c' or group == 'b'"), 
    cols("-rt")
    )


,score,speed,group,rt_doubled
0,0.436753,0.836273,b,1.137357
1,0.983424,0.809752,c,0.554110
2,0.404657,0.746343,b,0.168047
3,0.355559,0.434809,c,1.975218
4,0.819739,0.279964,b,1.066826
5,0.723915,0.948239,c,0.673403
6,0.427613,0.979953,b,0.876543
7,0.025963,0.677200,c,0.961849


#### ints

Both `rows` and `cols` interpret integers as *numerical indices* (i.e. `.iloc`):

In [6]:
# Second row, third column
pipe(df,
    rows(1),
    cols(2)
     )

,speed
1,0.836273


#### lists/arrays

Both `rows` and `colws` interprets lists and arrays as *numerical indices* (i.e. `.iloc`) if they contain numbers, but as *row/col labels* (i.e. `.loc`) if they contain strings.

In [7]:
pipe(df, 
    rows([0, 2, 4]), 
    cols(["rt", "speed", "score"])
    )


,rt,speed,score
0,0.600285,0.198946,0.696828
2,0.277055,0.809752,0.983424
4,0.062525,0.500357,0.764594


In [8]:
# Same but using indexes for columns
pipe(df, 
    rows([0, 2, 4]), 
    cols([0, 2, 1])
    )


,rt,speed,score
0,0.600285,0.198946,0.696828
2,0.277055,0.809752,0.983424
4,0.062525,0.500357,0.764594


#### tuples

Both `rows` and `cols` interpret tuples as `(start, stop, step)` slices, with `step` being optional:

In [9]:
pipe(df, 
     rows((1, 11, 2)), 
     cols((0, 5, 2)))


,rt,speed,rt_doubled
1,0.568678,0.836273,1.137357
3,0.193848,0.026424,0.387695
5,0.509818,0.099443,1.019636
7,0.987609,0.434809,1.975218
9,0.022993,0.565018,0.045986


In [10]:
pipe(df,
     rows((0, 5))
)

,rt,score,speed,group,rt_doubled
0,0.600285,0.696828,0.198946,a,1.200570
1,0.568678,0.436753,0.836273,b,1.137357
2,0.277055,0.983424,0.809752,c,0.554110
3,0.193848,0.632703,0.026424,d,0.387695
4,0.062525,0.764594,0.500357,e,0.125049


### Summarizing data
To perform an operation that results in a *scalar* output per column, use `summarize`. It always return a *smaller* dataframe or series than the original

Non-grouped inputs produce *Series* results:

In [7]:
from utilz import summarize

pipe(df, 
    rows("group == 'c' or group == 'b'"), 
    summarize(rt='mean',speed='mean')
    )

rt       0.695871
speed    0.542008
dtype: float64

Grouped inputs product *DataFrame* results:

In [8]:
from utilz import groupby

pipe(df, 
    groupby('group'), 
    summarize(score = 'mean', rt = 'std')
    )

,score,rt
group,,
a,0.504750,0.195151
b,0.629777,0.145344
c,0.374209,0.372188
d,0.474965,0.354329
e,0.498287,0.415910


### Transforming data
To perfom an operation that results in a *non-scalar* output per column, or to ensure that an operation with *scalar* output retains the original shape of the input, use `assign`. It always returns back a DataFrame the *same* size as the original via broadcasting.

With non-grouped inputs, the mean score is broadcasted across the *entire* DataFrame

In [6]:
pipe(df, 
     assign(mean_overall_score="score.mean()")
     )


,rt,score,speed,group,rt_doubled,mean_overall_score
0,0.902407,0.177135,0.792267,a,1.804814,0.556813
1,0.805787,0.337138,0.312966,b,1.611573,0.556813
2,0.517491,0.033439,0.934013,c,1.034982,0.556813
3,0.261900,0.833535,0.723541,d,0.523799,0.556813
4,0.637885,0.611255,0.257965,e,1.275769,0.556813
5,0.700008,0.348929,0.930422,a,1.400015,0.556813
6,0.858416,0.044372,0.346503,b,1.716832,0.556813
7,0.701888,0.891862,0.240973,c,1.403776,0.556813
8,0.854061,0.921804,0.227022,d,1.708123,0.556813
9,0.456034,0.297202,0.190947,e,0.912067,0.556813


With grouped inputs, the mean score is broadcasted *within* each group:

In [7]:
pipe(df, 
    groupby('group'), 
    assign(mean_grp_score = 'score.mean()')
    )

,rt,score,speed,group,rt_doubled,mean_grp_score
0,0.902407,0.177135,0.792267,a,1.804814,0.480874
1,0.805787,0.337138,0.312966,b,1.611573,0.484267
2,0.517491,0.033439,0.934013,c,1.034982,0.553818
3,0.261900,0.833535,0.723541,d,0.523799,0.663214
4,0.637885,0.611255,0.257965,e,1.275769,0.601892
5,0.700008,0.348929,0.930422,a,1.400015,0.480874
6,0.858416,0.044372,0.346503,b,1.716832,0.484267
7,0.701888,0.891862,0.240973,c,1.403776,0.553818
8,0.854061,0.921804,0.227022,d,1.708123,0.663214
9,0.456034,0.297202,0.190947,e,0.912067,0.601892


Here no broadcasting occurs, but the result is reshaped while respecting groups like `transform` in pandas.

In [8]:
pipe(df, 
    groupby('group'), 
    assign(
        score_centered='score - score.mean()', 
        score_norm = 'score/score.std()',
        score_zscore ='(score - score.mean()) / score.std()'
        )
        
    )

,rt,score,speed,group,rt_doubled,score_centered,score_norm,score_zscore
0,0.902407,0.177135,0.792267,a,1.804814,-0.303739,0.634311,-1.087670
1,0.805787,0.337138,0.312966,b,1.611573,-0.147128,0.846184,-0.369277
2,0.517491,0.033439,0.934013,c,1.034982,-0.520378,0.075783,-1.179330
3,0.261900,0.833535,0.723541,d,0.523799,0.170321,2.446669,0.499943
4,0.637885,0.611255,0.257965,e,1.275769,0.009363,2.859903,0.043809
5,0.700008,0.348929,0.930422,a,1.400015,-0.131945,1.249493,-0.472488
6,0.858416,0.044372,0.346503,b,1.716832,-0.439895,0.111369,-1.104092
7,0.701888,0.891862,0.240973,c,1.403776,0.338044,2.021220,0.766107
8,0.854061,0.921804,0.227022,d,1.708123,0.258590,2.705764,0.759038
9,0.456034,0.297202,0.190947,e,0.912067,-0.304689,1.390532,-1.425563


Assignments can be arbitrary complex as long as they can be expressed as a string. They can even make reference to other columns:

In [9]:
pipe(df,
    groupby('group'),
     assign(
         price="(rt > .5) * speed"
     )
)

,rt,score,speed,group,rt_doubled,price
0,0.902407,0.177135,0.792267,a,1.804814,0.792267
1,0.805787,0.337138,0.312966,b,1.611573,0.312966
2,0.517491,0.033439,0.934013,c,1.034982,0.934013
3,0.261900,0.833535,0.723541,d,0.523799,0.000000
4,0.637885,0.611255,0.257965,e,1.275769,0.257965
5,0.700008,0.348929,0.930422,a,1.400015,0.930422
6,0.858416,0.044372,0.346503,b,1.716832,0.346503
7,0.701888,0.891862,0.240973,c,1.403776,0.240973
8,0.854061,0.921804,0.227022,d,1.708123,0.227022
9,0.456034,0.297202,0.190947,e,0.912067,0.000000


You can also combine them with some handy helpers from `plydata`, but you'll have to pass the `use_ply=True` if you're using `groupby`:

In [19]:
from plydata import case_when

df = pipe(
    df,
    groupby("group", use_ply=True),
    assign(
        mean_grp_speed="speed.mean()",
        speed_tier=case_when(
            {
                'speed < speed.mean()': '"one"',
                'speed == speed.mean()': '"two"',
                'speed > speed.mean()': '"three"',
            }
        )
    ),
)
df

groups: ['group']
          rt     score     speed group  rt_doubled  mean_grp_speed speed_tier
0   0.835214  0.469788  0.818336     a    1.670428        0.689456      three
1   0.933498  0.803162  0.836482     b    1.866995        0.741970      three
2   0.870527  0.988006  0.045092     c    1.741054        0.342046        one
3   0.331299  0.188222  0.201379     d    0.662598        0.507731        one
4   0.060063  0.527094  0.873889     e    0.120126        0.613322      three
5   0.663402  0.772564  0.730931     a    1.326803        0.689456      three
6   0.881181  0.562835  0.373125     b    1.762361        0.741970        one
7   0.772267  0.104532  0.375590     c    1.544535        0.342046      three
8   0.145882  0.500106  0.343859     d    0.291764        0.507731        one
9   0.834803  0.905774  0.474066     e    1.669605        0.613322        one
10  0.697991  0.060713  0.765796     a    1.395981        0.689456      three
11  0.602443  0.370808  0.787415     b    1.204887        0.741970      three
12  0.663916  0.298299  0.302135     c    1.327832        0.342046        one
13  0.762938  0.904471  0.933878     d    1.525876        0.507731      three
14  0.203782  0.463020  0.827351     e    0.407565        0.613322      three
15  0.371209  0.715934  0.442762     a    0.742419        0.689456        one
16  0.799226  0.782305  0.970858     b    1.598452        0.741970      three
17  0.043911  0.105998  0.645368     c    0.087822        0.342046      three
18  0.898024  0.307063  0.551807     d    1.796048        0.507731      three
19  0.855099  0.097262  0.277984     e    1.710198        0.613322        one

In [25]:
pipe(df, 
     assign(
         relabel=case_when(
             {
                 "speed_tier == 'one'": "'winner'",
                 "speed_tier == 'two'": "'loser'",
                 "speed_tier == 'three'": "'loser'",
                 }
             )
         )
     )


groups: ['group']
          rt     score     speed group  rt_doubled  mean_grp_speed speed_tier  \
0   0.835214  0.469788  0.818336     a    1.670428        0.689456      three   
1   0.933498  0.803162  0.836482     b    1.866995        0.741970      three   
2   0.870527  0.988006  0.045092     c    1.741054        0.342046        one   
3   0.331299  0.188222  0.201379     d    0.662598        0.507731        one   
4   0.060063  0.527094  0.873889     e    0.120126        0.613322      three   
5   0.663402  0.772564  0.730931     a    1.326803        0.689456      three   
6   0.881181  0.562835  0.373125     b    1.762361        0.741970        one   
7   0.772267  0.104532  0.375590     c    1.544535        0.342046      three   
8   0.145882  0.500106  0.343859     d    0.291764        0.507731        one   
9   0.834803  0.905774  0.474066     e    1.669605        0.613322        one   
10  0.697991  0.060713  0.765796     a    1.395981        0.689456      three   
11  0.602443  0.370808  0.787415     b    1.204887        0.741970      three   
12  0.663916  0.298299  0.302135     c    1.327832        0.342046        one   
13  0.762938  0.904471  0.933878     d    1.525876        0.507731      three   
14  0.203782  0.463020  0.827351     e    0.407565        0.613322      three   
15  0.371209  0.715934  0.442762     a    0.742419        0.689456        one   
16  0.799226  0.782305  0.970858     b    1.598452        0.741970      three   
17  0.043911  0.105998  0.645368     c    0.087822        0.342046      three   
18  0.898024  0.307063  0.551807     d    1.796048        0.507731      three   
19  0.855099  0.097262  0.277984     e    1.710198        0.613322        one   

   relabel  
0    loser  
1    loser  
2   winner  
3   winner  
4    loser  
5    loser  
6   winner  
7    loser  
8   winner  
9   winner  
10   loser  
11   loser  
12  winner  
13   loser  
14   loser  
15  winner  
16   loser  
17   loser  
18   loser  
19  winner

## Efficient analyses using memoization, currying, and caching



It's convenient to combine these operations using `pipe`, but what if some operations take longer than others? It would be annoying to have to rerun expensive i/o or computate operations while you're debugging or adding another step to your pipeline. This section demonstrates three ways that can dramatically improve efficiency.

### Memoization

**Memoize** a function to save its last input in memory (RAM) and recall it when called with the same arguments rather than re-executing a potentially long running function. Memoized outputs *do not* persist across kernel restarts, and aren't great for functions that have very large or non-pickleable inputs and outputs. A nice use case is simply loading a file. Given the filepath, you're loading the same file each time, so lets just memoize the result, especially if it takes a long time to reread the file:

In [16]:
from toolz import memoize
from time import sleep

@memoize
def load(path):
    "Simulate slow loading a file..."
    print("loading from disk")
    sleep(5)
    return pd.read_csv(path)

### Currying
**Curry**, also called *partial function application*, is helpful when writing custom functions you want to add to a `pipe`. When curried, a function operates normally when it receives all its required arguments, but turns into a *partial* function when it gets fewer than all its required arguments. This partial function behaves just like the original except with a subset of its arguments "fixed". Because `pipe` implicitly passes the output of the last function to the next function in the pipeline, it's not possible to manipulate secondary arguments to a function in the pipeline without currying:

In [ ]:
from toolz import curry

@curry
def calc_mean(df, normalize=False):
    pass

# Now this works, otherwise it would complain about the wrong number of arguments
# to calc_mean
pipe(df,
    calc_mean(normalize=True)
)

# Note: with no kwargs you sometimes have to write the args backwards
@curry
def calc_mean(norm_value, df):
    pass


### Caching outputs to disk
**Cache** so the result of a function is stored to disk in a file made unique by hash of the args and kwargs to the function. Use `utilz.disk_cache` to decorate a function so it caches, which works similar to `toolz.memoize` but stores the result to a file (rather than in memory) and loads the file when called with the same inputs. Essentially you're trading computation time for i/o time. This is usually preferable to `memoize` for larger memory hungry inputs and outputs, and necessary if input or outputs cannot be pickled (e.g. dataframes, arrays, deep objects, etc). Setting the threshold to something like 1 essentially always caches the result. 

In [19]:
from utilz import disk_cache

# We're also using the curry decorator so norm works in the pipeline below
# It's not needed to use the disk_cache decorator

@curry
@disk_cache(threshold=1)
def norm(df, num='', denom=''):
    "Simulate expensive function that takes args"
    print("computing...")
    sleep(5)
    return pd.DataFrame({"norm": df[num] / df[denom]})

## Putting it together
### Pipeline based data analysis with caching and memoization

First you might want to clear any local cache, i.e. the `.utilz_cache` folder. Not neccessary, but important so you don't accidently load a disk-cached output without realizing. `utilz` will always let you know if it's caching or loading a cached result to help you.


In [21]:
from utilz import clear_cache

# Clear any existing cache
clear_cache()

Because we used the `memoize` decorator when defining `load` above, only the first run of this pipeline actually loads the data and incurs an i/o cost. Likewise because `norm` is decorated with `disk_cache` only the first run of norm (with the same prior pipeline outputs) incurs a compute cost.

In [22]:
summary = pipe(
    "test.csv",
    load,
    groupby("group"),
    assign(
        score_centered="score - score.mean()", 
        score_mean="score.mean()", 
        score_std="score.std()"),
    norm(num='rt',denom='score')
)
summary

loading from disk
computing...
Exceeded compute time. Result cached to .utilz_cache/norm___denom__score--df__9c405c0813bd657babdcd3583202ebea2a7a61819abdb234b93c59dab0d05fcc--num__rt.csv


,norm
0,0.876757
4,0.219101
8,1.803562
12,0.080123
16,1.397509
1,0.013055
5,0.293790
9,0.678440
13,1.044078
17,0.495721


Notice how no data loading or expensive computation needs to happen on a second run of this pipeline (no print messages aside from `utilz` telling you it's loading `norm` last cached result):

In [24]:
summary = pipe(
    "test.csv",
    load,
    groupby("group"),
    assign(
        score_centered="score - score.mean()", 
        score_mean="score.mean()", 
        score_std="score.std()"),
    norm(num='rt',denom='score')
)
summary

Returning norm cached result


,norm
0,0.876757
1,0.219101
2,1.803562
3,0.080123
4,1.397509
5,0.013055
6,0.293790
7,0.678440
8,1.044078
9,0.495721


## Summary


This setup is nice because it allows for both interactive data analysis as well as reproducible scripts. Simply start writing the pipeline steps, and comment out ones you want to skip or debug. In another notebook cell edit the source code of a function in the pipeline and incrementally add to its body, while rerunning the pipeline to see results as you build up your functions.

For functions that take a while to run, try decorating them with `memoize` or `disk_cache`. Memoize is nice for loading csv/text files (so you don't need to re-read them from disk each re-run of the pipeline). Cacheing is nice for expensive operations or operations on complex datastructures like arrays and dataframes. Plus, utilz saves them in standard robust file types (.csv. or .h5 rather than pickles) so you're also getting incremental backups of your work. No more need to rely on saved "state" in a Juptyer notebook.